# 03. Building a Chatbot with Memory

## Setting up

### LangSmith Tracing

After signing up at [LangSmith](https://smith.langchain.com/), make sure to set your environment variable to start logging traces: 
```bash 
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..." 
```

or, you can set them manually in a notebook

In [11]:
import getpass 
import os

os.environ["LANGSMITH_TRACING"] = "true" 
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


### Get `.env` variable for LLM API Key

In [12]:
from dotenv import load_dotenv

load_dotenv() 

gemini_api_key = os.getenv("GEMINI_API_KEY")

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found. Please set it up in the .env file")

### Initialize the model

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI 

model = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash", 
    temperature = 0, 
    api_key = gemini_api_key
)

In [16]:
from langchain_core.messages import HumanMessage 

model.invoke([HumanMessage(content="Hi!, I am Dipesh")])

AIMessage(content="Hi Dipesh! Nice to meet you.\n\nI'm an AI assistant. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8acfad19-f5ee-46e1-8056-6a92fdba00fa-0', usage_metadata={'input_tokens': 7, 'output_tokens': 488, 'total_tokens': 495, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 464}})

The model does not have any concept of state. If you ask a followup questions, it can't answer. For example, 

In [17]:
model.invoke([HumanMessage(content="What is my name?")]) 

AIMessage(content="I don't know your name. As an AI, I don't have access to personal information about you or any memory of past conversations.\n\nYou haven't told me your name. If you'd like me to refer to you by a name during this conversation, please feel free to tell me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3438491c-9aaf-444b-b0fc-de32bdccb976-0', usage_metadata={'input_tokens': 6, 'output_tokens': 514, 'total_tokens': 520, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 450}})

To fix this we need to pass the entire conversation history into the model. Let's see what happens when we do that: 

In [21]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Dipesh."), 
        AIMessage(content="Hello Dipesh! How can I assist you today?"), 
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Dipesh.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--fabf0854-8435-43ff-b45c-908de83eec2b-0', usage_metadata={'input_tokens': 28, 'output_tokens': 43, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 37}})

## Message Persistence

### Persistence

LangGraph has a built-in persistence layer, implemented through checkpointers. When you compile a graph with a checkpointer, the checkpointer saves a`checkpoint` of the graph state at every super-step. Those checkpoints are saved to a `thread`, which an be accessed after graph execution.

For more advanced use cases, LangGraph supports other persistence backends like SQLite or Postgres—check out the LangGraph persistence [documentation](https://langchain-ai.github.io/langgraph/concepts/persistence/?_gl=1*1f81ite*_gcl_au*MTM2NjY0NDk3My4xNzUzMTk5OTg5*_ga*MTA4ODYwNTkwNC4xNzUzMjM4NTE1*_ga_47WX3HKKY2*czE3NTM0MTM2MTkkbzQkZzEkdDE3NTM0MTM3NzEkajU3JGwwJGgw) for details.

![Persistence](https://langchain-ai.github.io/langgraph/concepts/img/persistence/checkpoints.jpg)

### Threads 

A thread is a unique ID or thread identifier assigned to each checkpoint saved by a checkpointer. It contains the accumulated state of a sequence of runs. When a run is executed, the state of the underlying graph of the assistant will be persisted to the thread.

When invoking graph with a checkpointer, you must specify a thread_id as part of the configurable portion of the config:
```python 
{"configurable": {"thread_id": "1"}}
``` 
A thread's current and historical state can be retrieved. To persist state, a thread must be created prior to executing a run.

### Checkpoints 

The state of a thread at a particular point in time is called a checkpoint. Checkpoint is a snapshot of the graph state saved at each super-step and is represented by StateSnapshot object with the following key properties:

- `config`: Config associated with this checkpoint.
- `metadata`: Metadata associated with this checkpoint.
- `values`: Values of the state channels at this point in time.
next A tuple of the node names to execute next in the graph.
- `tasks`: A tuple of `PregelTask` objects that contain information about next tasks to be executed. If the step was previously attempted, it will include error information. If a graph was interrupted [dynamically](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/add-human-in-the-loop/) from within a node, tasks will contain additional data associated with interrupts.
  
Checkpoints are persisted and can be used to restore the state of a thread at a later time.

In [54]:
from langgraph.checkpoint.memory import MemorySaver 
from langgraph.graph import START, MessagesState, StateGraph 

# Define a new graph 
workflow = StateGraph(state_schema=MessagesState) 

# Define the function that calls the model 
def call_model(state: MessagesState): 
    response = model.invoke(state["messages"]) 
    return {"messages": response} 

# Define the (single) node in the graph  
workflow.add_edge(START, "model") 
workflow.add_node("model", call_model) 

# Add memory 
memory = MemorySaver() 
app = workflow.compile(checkpointer=memory) 

# Create a config for including threadi_id 
config = {"configurable": {"thread_id": "abc123"}} 

We can then invoke the application 

### Example: message inputs

In [55]:
query = "Hi! I am Dipesh." 

input_messages = [HumanMessage(query)] 
output = app.invoke({"messages": input_messages}, config) 
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Dipesh! Nice to meet you.

How can I help you today?


In [56]:
query = "What's my name?" 
input_messages = [HumanMessage(query)] 
output = app.invoke({"messages": input_messages}, config) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Dipesh.


Now our chatbot remebers things abou us. If we change the config to reference a different `thread_id`. We can see that it starts the conversation fresh. 

In [57]:
config_2 = {"configurable": {"thread_id": "abc234"}} 

input_messages = [HumanMessage(query)] 
output = app.invoke({"messages": input_messages}, config=config_2) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As an AI, I don't have access to personal information about you, including your name. I don't store personal data or remember individual users from past interactions.

If you'd like me to refer to you by a specific name during our conversation, please feel free to tell me!


We can always go back to the original conversation as we are persisting it in a database. Let's pass the previous configurable `config` instead of `config_2`

In [58]:
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)  
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Dipesh.


### Async support

For async support, update the `call_model` node to be an async function and use `.ainvoke` when invoking the application. 

In [59]:
# ansync function for node: 
async def call_model(state: MessagesState): 
    response = await model.ainvoke(state["messages"]) 
    return {"messages": response}

# Define graph as before 
workflow = StateGraph(state_schema=MessagesState) 
workflow.add_edge(START, "model") 
workflow.add_node("model", call_model) 
app = workflow.compile(checkpointer=MemorySaver())

# App invocation 
output = await app.ainvoke({"messages": input_messages}, config) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As an AI, I don't have access to personal information about you, including your name. I don't store personal data or remember individual users from past interactions.

If you'd like me to refer to you by a specific name during our conversation, please feel free to tell me!


As this is a new graph, it doesn't have the state of the MemoryState of the previous graph.

### Example: dictionary inputs

LangChain runnables often accept multiple inputs via separate keys in a single dict argument. A common example is a prompt template with multiple parameters. 

Whereas before our runnable was a chat model, here we chain together a prompt template and chat model.

In [77]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder 

prompt = ChatPromptTemplate(
    [
        ("system", "Answer in {language}"), 
        MessagesPlaceholder(variable_name="messages"), 
    ]
) 

runnable = prompt | model 

We then define a sigle-node graph in the same way as before. 

In below state:  
- Updates to the `messages` list will append messages
- Updates to the `language` string will overwrite the string

In [78]:
from typing import Sequence 

from langchain_core.messages import BaseMessage 
from langgraph.graph.message import add_messages 
from typing_extensions import Annotated, TypedDict 

class State(TypedDict): 
    messages: Annotated[Sequence[BaseMessage], add_messages] 
    language: str 

workflow = StateGraph(state_schema=State) 

def call_model(state: State): 
    response = runnable.invoke(state) 
    # update the messsage history with response 
    return {"messages": [response]} 

workflow.add_edge(START, "model") 
workflow.add_node("model", call_model) 

memory = MemorySaver() 
app = workflow.compile(checkpointer=memory) 

In [79]:
config = {"configurable": {"thread_id": "abc345"}} 

input_dict = {
    "messages": [HumanMessage("Hi, I'm Dipesh.")], 
    "language": "Nepali",
} 
output = app.invoke(input_dict, config) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते, दिपेश जी।


Let's have a few more conversations

In [83]:
config = {"configurable": {"thread_id": "abc345"}} 

input_dict = {
    "messages": [HumanMessage("Hi, I'm Dipesh.")], 
    "language": "Spanish",
} 
output = app.invoke(input_dict, config) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hola, Dipesh. ¿En qué puedo ayudarte hoy?


In [85]:
config = {"configurable": {"thread_id": "abc345"}} 

input_dict = {
    "messages": [HumanMessage("Hi, I'm Dipesh.")], 
    "language": "Hindi",
} 
output = app.invoke(input_dict, config) 
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते दिपेश। मैं आपकी क्या सहायता कर सकता हूँ?


## Managing message history

### Get state 
The message history (and other elements of the application state) can be accessed via `.get_state`. 

When interacting with the saved graph state, you must specify a thread identifier. You can view the latest state of the graph by calling `graph.get_state(config)`. This will return a `StateSnapshot` object that corresponds to the latest checkpoint associated with the thread ID provided in the config or a checkpoint associated with a checkpoint ID for the thread, if provided.

In [92]:
state = app.get_state(config).values
state

{'messages': [HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='ae17fed3-2a13-4b42-8858-d1b47eb10025'),
  AIMessage(content='नमस्ते, दिपेश जी।', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2cbcf658-6bcf-4d4e-be6e-31afb3e5cf6c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 224, 'total_tokens': 236, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 218}}),
  HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='95647078-a5f3-422f-9b7a-dcfd19b111b3'),
  AIMessage(content='Hola, Dipesh. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--52162666-377e-489d-b45d-89

In [90]:
print(f"Language:{state['language']}")  
for message in state["messages"]: 
    message.pretty_print() 

Language:Hindi
================================ Human Message =================================

Hi, I'm Dipesh.
================================== Ai Message ==================================

नमस्ते, दिपेश जी।
================================ Human Message =================================

Hi, I'm Dipesh.
================================== Ai Message ==================================

Hola, Dipesh. ¿En qué puedo ayudarte hoy?
================================ Human Message =================================

Hi, I'm Dipesh.
================================== Ai Message ==================================

नमस्ते दिपेश। मैं आपकी क्या सहायता कर सकता हूँ?


Get a `StateSnapshot` for a specific `checkpiont_id`

In [97]:
config = {"configurable": {"thread_id": "abc345", "checkpoint_id":"run--52162666-377e-489d-b45d-89885e0264eb-0" }} 
state = app.get_state(config)
state

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': 'abc345', 'checkpoint_id': 'run--52162666-377e-489d-b45d-89885e0264eb-0'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

### Get state history 

You can get the full history of the graph execution for a given thread by calling `graph.get_state_history(config)`. This will return a list of `StateSnapshot` objects associated with the thread ID provided in the config. Importantly, the checkpoints will be ordered chronologically with the most recent `checkpoint` / `StateSnapshot` being the first in the list.

In [107]:
config = {"configurable": {"thread_id": "abc345"}}
list(app.get_state_history(config)) 

[StateSnapshot(values={'messages': [HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='ae17fed3-2a13-4b42-8858-d1b47eb10025'), AIMessage(content='नमस्ते, दिपेश जी।', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2cbcf658-6bcf-4d4e-be6e-31afb3e5cf6c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 224, 'total_tokens': 236, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 218}}), HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='95647078-a5f3-422f-9b7a-dcfd19b111b3'), AIMessage(content='Hola, Dipesh. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--52162666-3

![image](https://langchain-ai.github.io/langgraph/concepts/img/persistence/get_state.jpg)

### Update state 

We can also update the state via .update_state. For example, we can manually append a new message:


In [108]:
from langchain_core.messages import HumanMessage 

_ = app.update_state(config, {"messages": [HumanMessage("Test")]}) 

In [111]:
state = app.get_state(config).values 

print(f"Language: {state["messages"]}") 
for message in state["messages"]: 
    message.pretty_print()

Language: [HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='ae17fed3-2a13-4b42-8858-d1b47eb10025'), AIMessage(content='नमस्ते, दिपेश जी।', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2cbcf658-6bcf-4d4e-be6e-31afb3e5cf6c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 224, 'total_tokens': 236, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 218}}), HumanMessage(content="Hi, I'm Dipesh.", additional_kwargs={}, response_metadata={}, id='95647078-a5f3-422f-9b7a-dcfd19b111b3'), AIMessage(content='Hola, Dipesh. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--52162666-377e-489d-b45d-89885e0264e

## Checkpointer libraries¶

Under the hood, checkpointing is powered by checkpointer objects that conform to [BaseCheckpointSaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver) interface. LangGraph provides several checkpointer implementations, all implemented via standalone, installable libraries:

- `langgraph-checkpoint`: The base interface for checkpointer savers (BaseCheckpointSaver) and serialization/deserialization interface ([SerializerProtocol](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.serde.base.SerializerProtocol)). Includes in-memory checkpointer implementation ([InMemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.InMemorySaver)) for experimentation. LangGraph comes with `langgraph-checkpoint` included.
- `langgraph-checkpoint-sqlite`: An implementation of LangGraph checkpointer that uses SQLite database ([SqliteSaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.sqlite.SqliteSaver) / [AsyncSqliteSaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.sqlite.aio.AsyncSqliteSaver)). Ideal for experimentation and local workflows. Needs to be installed separately.
- `langgraph-checkpoint-postgres`: An advanced checkpointer that uses Postgres database ([PostgresSaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.postgres.PostgresSaver) / [AsyncPostgresSaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.postgres.aio.AsyncPostgresSaver)), used in LangGraph Platform. Ideal for using in production. Needs to be installed separately.

### Checkpointer interface
Each checkpointer conforms to BaseCheckpointSaver interface and implements the following methods:

- `.put` - Store a checkpoint with its configuration and metadata.
- `.put_writes` - Store intermediate writes linked to a checkpoint (i.e. pending writes).
- `.get_tuple` - Fetch a checkpoint tuple using for a given configuration (`thread_id` and `checkpoint_id`). This is used to populate `StateSnapshot` in `graph.get_state()`.
- `.list` - List checkpoints that match a given configuration and filter criteria. This is used to populate state history in `graph.get_state_history()`.

If the checkpointer is used with asynchronous graph execution (i.e. executing the graph via `.ainvoke`, `.astream`, `.abatch`), asynchronous versions of the above methods will be used (`.aput`, `.aput_writes`, `.aget_tuple`, `.alist`).

## References 

- LangChain - [Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)
- LangGraph Documentation - [Persistence](https://python.langchain.com/docs/tutorials/chatbot/)
- LangChain - [How to add messages history?](https://python.langchain.com/docs/how_to/message_history/)
